# Lab 10 (March 23): Compressive Sensing

In [1]:
import numpy as np
from numpy import linalg as LA
import warnings
warnings.filterwarnings('ignore')

Rough Explaination of the success of $\ell_1$ minimization:

Theorem 14.2 
(1) $\ell_1$ minimization is successful;
(2) matrix $A$ satisfies null space property

Small coherence of $A$ implies the null space property, however, we need $m\gtrsim s^2$ to met small coherence property.

So, we introduce Restricted Isometry Property. There are $\ell_1$-Restricted Isometry Property and $\ell_2$-Restricted Isometry Property. 

Random matrices satisfy Restricted Isometry Property with high probability.

In [2]:
# generate a sparse vector together with its observation vector
N = 2000             # ambient dimension
m = 500              # number of measurements
s = 80               # sparsity level
# create the sparse vector x
x = np.zeros(N)
supp = np.sort(np.random.permutation(N)[:s])
x[supp] = np.random.normal(0,1,s)
# define the observation matrix A (Theorem 14.6 to satisfy RIP)
A = np.sqrt(np.pi/2)/m*np.random.normal(0,1,size=(m,N))
# produce the observation vector y
y = A@x

In [3]:
import cvxpy as cp

#### $\ell_1$ minimization by directly using CVXPY
The $\ell_1$ minimizatio program:
$$ \mathop{\mathrm{minimize}}_{z\in\mathbb{R}^N} \|z\|_1 \quad \mbox{ s.to } Az=y $$

In [4]:
z = cp.Variable(N)
objective = cp.Minimize(cp.norm(z,1))
constraints = [A@z==y]
M1 = cp.Problem(objective,constraints)
M1.solve()
z = z.value
print(LA.norm(x-z)/LA.norm(x))
print(sum(z>1e-5))

5.400160140282881e-14
28


#### $\ell_1$ minimization to the following linear programming, then use CVXPY
by introducing nonnegative slack vectors $z^+$ and $z^-$, so as to solve the standard-form linear program $$ \underset{z^+,z^-\in\mathbb{R}^N}{\rm minimize} \sum_{j=1}^N (z^+_j + z^-_j) \qquad \mbox{subject to } A(z^+-z^-) = y, \; z^+ \ge 0, \; z^- \ge 0; $$

In [5]:
z1 = cp.Variable(N,pos=True)
z2 = cp.Variable(N,pos=True)
objective = cp.Minimize(cp.sum(z1+z2))
constraints = [A@(z1-z2)==y]
M2 = cp.Problem(objective,constraints)
M2.solve()
z = z1.value - z2.value
print(LA.norm(x-z)/LA.norm(x))
print(sum(z>1e-5))

2.2014539852387352e-10
28


#### $\ell_1$ minimization to the following linear programming, then use CVXPY
by introducing a slack vector $c$, so as to solve the alternative linear program $$ \underset{z,c\in\mathbb{R}^N}{\rm minimize} \sum_{j=1}^N c_j \qquad \mbox{subject to } Az = y, \; -c \leq z \leq c. $$

In [6]:
c = cp.Variable(N)
z = cp.Variable(N)
objective = cp.Minimize(cp.sum(c))
constraints = [A@z==y]
constraints+= [z+c>=0]
constraints+= [c-z>=0]
M3 = cp.Problem(objective,constraints)
M3.solve()
z = z.value
print(LA.norm(x-z)/LA.norm(x))
print(sum(z>1e-5))

5.400160140282881e-14
28


#### $\ell_1$ minimization to linear programming, then use scipy linear programming

$$ \underset{z^+,z^-\in\mathbb{R}^N}{\rm minimize} \sum_{j=1}^N (z^+_j + z^-_j) \qquad \mbox{subject to } A(z^+-z^-) = y, \; z^+ \ge 0, \; z^- \ge 0; $$

Documentation of scipy.optimize.linprog: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

In [7]:
from scipy.optimize import linprog

In [9]:
c = np.ones(2*N)
Aeq = np.column_stack((A,-A))
bounds = [(0,None)]*(2*N)   # it's not necessary, check the default bounds
res = linprog(c, A_eq=Aeq, b_eq=y,bounds=bounds, method='interior-point') 
z_plus = res.x[0:N]
z_minus = res.x[N:2*N]
z = z_plus-z_minus

In [11]:
print(LA.norm(x-z)/LA.norm(x))
print(sum(z>0))

4.591465514259358e-11
28


Play with method in linear programming.

Conclusion: Simplex method gives exact zero and interior-point method doesn't give exact zero